In [ ]:
# Импорты
import os
import csv
import cv2
import json
import math
import pandas as pd
from time import time
from tqdm import tqdm
from itertools import product
import vio_ort_exp as vio_ort

In [ ]:
# Путь к директории с данными
set_dir = '2024_12_15_15_31_8_num_3'

In [ ]:
# Чтение JSON-файлов
json_files = sorted([f for f in os.listdir(set_dir) if f.endswith('.json')])
start = 0  # Индекс начала обработки
count_json = len(json_files)  # Количество обрабатываемых файлов

In [ ]:
# Инициализация визуальной инерциальной одометрии (VIO)
odometry = vio_ort.VIO(lat0=54.889668, lon0=83.1258973333, alt0=0)

In [ ]:
# Определение параметров для экспериментов
top_k_values = [512, 256]  # Максимальное количество точек для сопоставления
detection_threshold_values = [0.05, 0.01]  # Порог детектирования ключевых точек
max_iters_values = [None, 100, 300, 500]  # Максимальное количество итераций оптимизации
rotation_methods = ["PIL", "CV2"]  # Методы поворота изображений
trace_values = [8, 4]  # Глубина трассировки

In [ ]:
# Генерация всех возможных комбинаций параметров
parameters = [
    {'top_k': top_k, 'detection_threshold': detection_threshold, 'maxIters': max_iters, 'rotation': rotation, 'trace': trace}
    for top_k, detection_threshold, max_iters, rotation, trace in product(
        top_k_values, detection_threshold_values, max_iters_values, rotation_methods, trace_values
    )
]

In [ ]:
def run_vio(odometry, json_files, start, count_json, top_k, detection_threshold, maxIters, rotation, trace, is_gps=True):
    """
    Выполняет обработку данных VIO с заданными параметрами.
    
    Параметры:
        odometry: Объект VIO.
        json_files: Список JSON-файлов.
        start: Индекс начала обработки.
        count_json: Количество файлов для обработки.
        top_k: Максимальное количество ключевых точек.
        detection_threshold: Порог детектирования.
        maxIters: Максимальное количество итераций.
        rotation: Метод поворота изображения.
        trace: Глубина трассировки.
        is_gps: Флаг использования данных GPS.
    
    Возвращает:
        Словарь с координатами VIO и GPS.
    """
    lat_VIO, lon_VIO, alt_VIO = [], [], []
    lat_GPS, lon_GPS, alt_GPS = [], [], []

    odometry._matcher.top_k = top_k
    odometry._matcher.detection_threshold = detection_threshold
    odometry.MAX_ITERS = maxIters
    odometry.ROTATION = rotation
    odometry.TRACE = trace

    for filename in json_files[start:start + count_json]:
        with open(f'{set_dir}/{filename}', 'r') as file:
            data = json.load(file)
            if 'GNRMC' in data and data['GNRMC']['status'] == 'A':
                img_path = os.path.join(set_dir, os.path.splitext(filename)[0] + '.jpg')
                image = cv2.imread(img_path)
                result_vio = odometry.add_trace_pt(image, data)

                lat_VIO.append(result_vio['lat'])
                lon_VIO.append(result_vio['lon'])
                alt_VIO.append(result_vio['alt'] * 1000)
                
                if is_gps:
                    lat_GPS.append(data['GNRMC'].get('lat', 0.0))
                    lon_GPS.append(data['GNRMC'].get('lon', 0.0))
                    alt_GPS.append(data['GPS_RAW_INT']['alt'])

    return {
        'lat_VIO': lat_VIO,
        'lon_VIO': lon_VIO,
        'alt_VIO': alt_VIO,
        'lat_GPS': lat_GPS,
        'lon_GPS': lon_GPS,
        'alt_GPS': alt_GPS,
    }

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Вычисляет расстояние между двумя точками на поверхности Земли.
    
    Параметры:
        lat1, lon1: Координаты первой точки.
        lat2, lon2: Координаты второй точки.
    
    Возвращает:
        Расстояние в метрах.
    """
    R = 6371000  # Радиус Земли в метрах
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [ ]:
def calculate_aggregated_metrics(results):
    """
    Рассчитывает агрегированные метрики (максимальное, минимальное расстояние, RMSE).
    
    Параметры:
        results: Словарь с координатами VIO и GPS.
    
    Возвращает:
        Словарь с метриками.
    """
    metrics = {"VIO": []}

    for i in range(len(results['lat_GPS'])):
        gps_lat, gps_lon = results['lat_GPS'][i], results['lon_GPS'][i]
        vio_lat, vio_lon = results['lat_VIO'][i], results['lon_VIO'][i]

        metrics["VIO"].append(haversine(gps_lat, gps_lon, vio_lat, vio_lon))

    distances = metrics["VIO"]
    aggregated = {
        "Metric": ["Max Distance (m)", "Min Distance (m)", "RMSE (m)"],
        "VIO": [max(distances), min(distances), math.sqrt(sum(d ** 2 for d in distances) / len(distances))]
    }

    return aggregated

In [ ]:
def save_results_to_csv(results_all, filename):
    """
    Сохраняет результаты экспериментов в файл CSV.
    
    Параметры:
        results_all: Список результатов экспериментов.
        filename: Имя выходного CSV-файла.
    """
    with open(filename, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Top_k", "Detection Threshold", "Max Iters", "Rotation method", "Trace depth", "Max Distance", "Min Distance", "RMSE"])

        for result in results_all:
            params = result['params']
            metrics = calculate_aggregated_metrics(result['results'])['VIO']
            writer.writerow([
                params['top_k'], params['detection_threshold'], params['maxIters'] if params['maxIters'] is not None else 'None', params['rotation'], params['trace'],
                metrics[0], metrics[1], metrics[2]
            ])

In [ ]:
# Запуск экспериментов
results_all = []
with tqdm(total=len(parameters), desc="Processing experiments", unit="experiment") as pbar:
    for params in parameters:
        start_time = time()
        results = run_vio(odometry, json_files, start, count_json, **params)
        results_all.append({'params': params, 'results': results})
        pbar.update(1)

In [ ]:
# Сохранение результатов
save_results_to_csv(results_all, "vio_parameters_comparison.csv")

In [ ]:
# Анализ и добавление FPS
file_name = 'vio_parameters_comparison.csv'
df = pd.read_csv(file_name)
df['Max Iters'] = df['Max Iters'].fillna('None')
df['Avg FPS'] = None

for idx, row in df.iterrows():
    print(f"Parameter set #{idx + 1}/{len(df)}")
    fps_values = input("Enter FPS values (e.g., 5.3, 6.7): ")
    fps_list = [float(fps) for fps in fps_values.split(',')]
    df.at[idx, 'Avg FPS'] = sum(fps_list) / len(fps_list)

df.drop(columns=['Time'], inplace=True)
df.to_csv('vio_results_updated.csv', index=False)
print("Results saved to 'vio_results_updated.csv'")